<a href="https://colab.research.google.com/github/PharahMain/Flight-Delay-Predictor/blob/master/flightstats_client.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz

In [2]:
# Install spark-related depdencies for Python
!pip install -q findspark
!pip install pyspark

     |████████████████████████████████| 215.7MB 57kB/s 
     |████████████████████████████████| 204kB 42.7MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130388 sha256=e67b4172bd8afadb5e1d7255614e142ec1e7be0bdabce3b0d502dd1059ebc0ab
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


In [0]:
# Set up required environment variables

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [4]:
# Point Colaboratory to Google Drive

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.sql import functions as F
from pyspark.sql.functions import isnan, when, count, col, create_map, lit
from pyspark.sql.functions import month, dayofweek, weekofyear, hour, minute
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.types import DateType, TimestampType

from datetime import datetime, date, timedelta
from itertools import chain
import random
import time

In [0]:
#ARR_JSON_PATH = f"/content/gdrive/My Drive/Colab Datasets/flights_data/arrivals/{date.today()}.json"
# for testing
ARR_JSON_PATH = "/content/gdrive/My Drive/Colab Datasets/flights_data/arrivals/2020-01-17.json"

#DEP_JSON_PATH = f"/content/gdrive/My Drive/Colab Datasets/flights_data/departures/{date.today()}.json"
DEP_JSON_PATH = f"/content/gdrive/My Drive/Colab Datasets/flights_data/departures/2020-01-17.json"

#WEATH_JSON_PATH = f"/content/gdrive/My Drive/Colab Datasets/flights_data/weather/{date.today()}.json"
WEATH_JSON_PATH = f"/content/gdrive/My Drive/Colab Datasets/flights_data/weather/2020-01-17.json"

APP_NAME = 'Flight Delay Predictor'
SPARK_URL = 'local[*]'
RANDOM_SEED = 263646
TRAINING_DATA_RATIO = 0.7

In [0]:
spark = SparkSession.builder.appName(APP_NAME).master(SPARK_URL).getOrCreate()

In [43]:
df_arr = spark.read.json(ARR_JSON_PATH)
df_arr.show(5)

+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+----------------------+--------------------+---------------------+--------------------+------------------+----------+------------+--------------------+--------------------+------+
|    airportResources|arrivalAirportFsCode|         arrivalDate|carrierFsCode|          codeshares|              delays|departureAirportFsCode|       departureDate|divertedAirportFsCode|     flightDurations|   flightEquipment|  flightId|flightNumber|    operationalTimes|            schedule|status|
+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+----------------------+--------------------+---------------------+--------------------+------------------+----------+------------+--------------------+--------------------+------+
|      [, TBIT,, C9,]|                 LAX|[2020-01-17T13:10...|           QR|     [[3515, UL, L]]| 

The data in a little more detail.

In [0]:
#df_arr.head(5)

Let's see how many flights arrived at LAX in the past 24 hours.

In [10]:
df_arr.count()

337

Let's first drop all the unnecessary columns.

In [0]:
df_arr = df_arr.drop('airportResources', 'codeshares', 'departureDate',
            'divertedAirportFsCode', 'operationalTimes', 'schedule', 'status',
            'flightId', 'flightNumber')
#df_arr.show(5)

In [12]:
df_arr.printSchema()

root
 |-- arrivalAirportFsCode: string (nullable = true)
 |-- arrivalDate: struct (nullable = true)
 |    |-- dateLocal: string (nullable = true)
 |    |-- dateUtc: string (nullable = true)
 |-- carrierFsCode: string (nullable = true)
 |-- delays: struct (nullable = true)
 |    |-- arrivalGateDelayMinutes: long (nullable = true)
 |    |-- arrivalRunwayDelayMinutes: long (nullable = true)
 |    |-- departureGateDelayMinutes: long (nullable = true)
 |    |-- departureRunwayDelayMinutes: long (nullable = true)
 |-- departureAirportFsCode: string (nullable = true)
 |-- flightDurations: struct (nullable = true)
 |    |-- blockMinutes: long (nullable = true)
 |    |-- scheduledAirMinutes: long (nullable = true)
 |    |-- scheduledBlockMinutes: long (nullable = true)
 |    |-- scheduledTaxiInMinutes: long (nullable = true)
 |    |-- scheduledTaxiOutMinutes: long (nullable = true)
 |    |-- taxiOutMinutes: long (nullable = true)
 |-- flightEquipment: struct (nullable = true)
 |    |-- actualEq

Let's explore the dataframe a little bit here.

In [13]:
df_arr.groupBy('carrierFsCode').count().orderBy('count', ascending=False).show()

+-------------+-----+
|carrierFsCode|count|
+-------------+-----+
|           DL|   48|
|           AA|   45|
|           OO|   44|
|           WN|   42|
|           CP|   30|
|           UA|   25|
|           AS|   24|
|           NK|    6|
|           B6|    6|
|           QX|    5|
|           AC|    4|
|           Y4|    3|
|           WS|    3|
|           BA|    3|
|           QF|    3|
|           K4|    2|
|           BR|    2|
|          BAI|    2|
|           NZ|    2|
|           1I|    2|
+-------------+-----+
only showing top 20 rows



Let's see what the busiest time of the day is for LAX next. But before we do that, I'm going to separate out the arrival time from 'arrivalDate'.

In [0]:
# first need to extract the date info from the arrivalDate struct
df_arr = df_arr.withColumn('localDate', col('arrivalDate').getField('dateLocal'))
#df_arr.select('localDate').show(5)

df_arr = df_arr.withColumn('localTimeStamp', df_arr['localDate'].cast(TimestampType()))
#df_arr.select('localTimeStamp').show(5)

Give each time increment its own column.

In [0]:
#df_arr.withColumn('date', col('localTimeStamp').cast('date')).withColumn('hour', hour(col('localTimeStamp'))).withColumn('minute', minute(col('localTimeStamp'))).show(5)

Since travel tends to change with the time of year, let's also add on weekOfYear and dayOfWeek as well.

In [0]:
#df_arr.withColumn('weekOfYear', weekofyear(col('localTimeStamp'))).withColumn('dayOfWeek', dayofweek(col('localTimeStamp'))).show(5)

Delays column, the column that contains our target data, is an array of four values. They are minutes of delay in different stages of the flight but, since travelers don't really care where the delays occur, but rather how long the delays are, let's sum up the numbers to get one total delayed minutes.

In [0]:
# a df with just the delay data
delay_df = df_arr.select('delays.*').na.fill(0)
#delay_df.show(5)

In [0]:
# create a new column in main arrival df with total delay minutes
delay_df = delay_df.withColumn('totalDelay', sum(delay_df[col] for col in delay_df.columns))
delay_df = delay_df.select('*').withColumn('d_id', monotonically_increasing_id())

#delay_df.show(5)


Now let's merge this table with the main table.

In [0]:
# In order to join two dataframes, we need to create an index column to join the two 
# dataframes on
df_arr = df_arr.select('*').withColumn('id', monotonically_increasing_id())

df_arr = df_arr.join(delay_df, col('id') == col('d_id'), how='leftouter').drop('d_id')
#df_arr.show(5)

Now let's do the same for flightDurations and flightEquipment.

In [0]:
eq_df = df_arr.select('flightEquipment.*').drop('tailNumber')
#eq_df.show(5)

When it comes to equipment, when it is changed it might cause a delay so I'm going to drop the tailNumber and create another column that shows whether there was a change in equipment.

In [0]:
eq_df = eq_df.withColumn('eqChanged', F.when(F.col('actualEquipmentIataCode') == F.col('scheduledEquipmentIatacode'), 0)
    .otherwise(1))

eq_df = eq_df.select('*').withColumn('e_id', monotonically_increasing_id())
#eq_df.show(5)

And lastly, let's just some up the flight durations.

In [0]:
fd_df = df_arr.select('flightDurations.*').na.fill(0)
fd_df = fd_df.withColumn('totalDuration', sum(fd_df[col] for col in fd_df.columns))
fd_df = fd_df.select('*').withColumn('f_id', monotonically_increasing_id())
#fd_df.show(5)

Put all the tables together.

In [0]:
df_arr = df_arr.join(eq_df, col('id') == col('e_id'), how='leftouter').drop('e_id').join(fd_df, col('id') == col('f_id'), how='leftouter').drop('f_id')
#df_arr.orderBy('id').show(5)

In [25]:
df_arr.count()

337

We now need to repeat the same process for the departing flights data. It might be better if I wrote a function that does this given a dataframe.

In [0]:
def format_flight_data(df):
    # lets' first index the df since we know we will be needing it later
    df = df.select('*').withColumn('id', monotonically_increasing_id())

    ##### timestamp #####
    # first need to extract the date info from the arrivalDate struct
    df = df.withColumn('localDate', col('arrivalDate').getField('dateLocal'))
    df = df.withColumn('localTimeStamp', df['localDate'].cast(TimestampType()))
    df = df.withColumn('date', col('localTimeStamp').cast('date'))
    df = df.withColumn('hour', hour(col('localTimeStamp')))
    df = df.withColumn('minute', minute(col('localTimeStamp')))
    df = df.withColumn('weekOfYear', weekofyear(col('localTimeStamp')))
    df = df.withColumn('dayOfWeek', dayofweek(col('localTimeStamp')))

    ##### delays #####
    # fill all null values with 0
    delay_df = df.select('delays.*').na.fill(0)
    # create a new column in main arrival df with total delay minutes
    delay_df = delay_df.withColumn('totalDelay', sum(delay_df[col] for col in delay_df.columns))
    delay_df = delay_df.select('*').withColumn('d_id', monotonically_increasing_id())

    ##### equipment #####
    eq_df = df.select('flightEquipment.*').drop('tailNumber')
    eq_df = eq_df.withColumn('eqChanged', F.when(F.col('actualEquipmentIataCode') == F.col('scheduledEquipmentIatacode'), 0).otherwise(1))
    eq_df = eq_df.select('*').withColumn('e_id', monotonically_increasing_id())

    ##### duration #####
    fd_df = df.select('flightDurations.*').na.fill(0)
    fd_df = fd_df.withColumn('totalDuration', sum(fd_df[col] for col in fd_df.columns))
    fd_df = fd_df.select('*').withColumn('f_id', monotonically_increasing_id())

    # join all dataframes
    formatted_df = df.join(delay_df, col('id')==col('d_id'), how='leftouter').drop('d_id')
    formatted_df = formatted_df.join(eq_df, col('id')==col('e_id'), how='leftouter').drop('e_id')
    formatted_df = formatted_df.join(fd_df, col('id')==col('f_id'), how='leftouter').drop('f_id')

    # drop extraneous columns
    formatted_df = formatted_df.drop('airportResources', 'codeshares', 'departureDate',
            'divertedAirportFsCode', 'operationalTimes', 'schedule', 'status',
            'flightId', 'flightNumber')

    return formatted_df

    

In [44]:
df_arr_fmtd = format_flight_data(df_arr)
df_arr_fmtd.show(5)

+--------------------+--------------------+-------------+--------------------+----------------------+--------------------+------------------+---+--------------------+-------------------+----------+----+------+----------+---------+-----------------------+-------------------------+-------------------------+---------------------------+----------+-----------------------+--------------------------+---------+------------+-------------------+---------------------+----------------------+-----------------------+--------------+-------------+
|arrivalAirportFsCode|         arrivalDate|carrierFsCode|              delays|departureAirportFsCode|     flightDurations|   flightEquipment| id|           localDate|     localTimeStamp|      date|hour|minute|weekOfYear|dayOfWeek|arrivalGateDelayMinutes|arrivalRunwayDelayMinutes|departureGateDelayMinutes|departureRunwayDelayMinutes|totalDelay|actualEquipmentIataCode|scheduledEquipmentIataCode|eqChanged|blockMinutes|scheduledAirMinutes|scheduledBlockMinutes|s

Nice. Now let's format the departing dataframe.

In [45]:
df_dep = spark.read.json(DEP_JSON_PATH)
df_dep_fmtd = format_flight_data(df_dep)
df_dep_fmtd.show(5)

+--------------------+--------------------+-------------+----------------+----------------------+-----------------+------------------+---+--------------------+-------------------+----------+----+------+----------+---------+-----------------------+-------------------------+-------------------------+---------------------------+----------+-----------------------+--------------------------+---------+-------------------+---------------------+----------------------+-----------------------+-------------+
|arrivalAirportFsCode|         arrivalDate|carrierFsCode|          delays|departureAirportFsCode|  flightDurations|   flightEquipment| id|           localDate|     localTimeStamp|      date|hour|minute|weekOfYear|dayOfWeek|arrivalGateDelayMinutes|arrivalRunwayDelayMinutes|departureGateDelayMinutes|departureRunwayDelayMinutes|totalDelay|actualEquipmentIataCode|scheduledEquipmentIataCode|eqChanged|scheduledAirMinutes|scheduledBlockMinutes|scheduledTaxiInMinutes|scheduledTaxiOutMinutes|totalDura

Now that we have a formatted dataframe for both arriving and departing flights, we need to join the weather conditions for each flight. The data from the DarkSky API contains within it a timestamp but it is in UNIX time. So now we just need to convert it so that it matches up with the formatted dataframe and do an inner join.

In [46]:
df_weath = spark.read.json(WEATH_JSON_PATH)
df_weath.show(5)

+-------------------+----------+--------+--------+-------------------+-----+---------------+-----------------+----------+--------+-------------------+-----------+----------+-------+----------+-----------+--------+---------+
|apparentTemperature|cloudCover|dewPoint|humidity|               icon|ozone|precipIntensity|precipProbability|precipType|pressure|            summary|temperature|      time|uvIndex|visibility|windBearing|windGust|windSpeed|
+-------------------+----------+--------+--------+-------------------+-----+---------------+-----------------+----------+--------+-------------------+-----------+----------+-------+----------+-----------+--------+---------+
|              53.33|      0.93|   48.75|    0.84|               rain|306.5|         0.0474|             0.87|      rain|  1020.0|         Light Rain|      53.33|1579248000|      0|      4.94|        118|    9.85|     7.89|
|               53.2|      0.88|   49.85|    0.88|               rain|303.5|         0.0262|            

In [47]:
df_weath = df_weath.withColumn('f_time', df_weath['time'].cast(TimestampType()))
df_weath.show(5)

+-------------------+----------+--------+--------+-------------------+-----+---------------+-----------------+----------+--------+-------------------+-----------+----------+-------+----------+-----------+--------+---------+-------------------+
|apparentTemperature|cloudCover|dewPoint|humidity|               icon|ozone|precipIntensity|precipProbability|precipType|pressure|            summary|temperature|      time|uvIndex|visibility|windBearing|windGust|windSpeed|             f_time|
+-------------------+----------+--------+--------+-------------------+-----+---------------+-----------------+----------+--------+-------------------+-----------+----------+-------+----------+-----------+--------+---------+-------------------+
|              53.33|      0.93|   48.75|    0.84|               rain|306.5|         0.0474|             0.87|      rain|  1020.0|         Light Rain|      53.33|1579248000|      0|      4.94|        118|    9.85|     7.89|2020-01-17 08:00:00|
|               53.2|   

Hmmm. Since the weather data is taken hourly, we now just need to match upto the hour.

In [48]:
# separate date and hour
df_weath = df_weath.withColumn('w_date', col('f_time').cast('date'))
df_weath = df_weath.withColumn('w_hour', hour(col('f_time')))
df_weath.show(5)

+-------------------+----------+--------+--------+-------------------+-----+---------------+-----------------+----------+--------+-------------------+-----------+----------+-------+----------+-----------+--------+---------+-------------------+----------+------+
|apparentTemperature|cloudCover|dewPoint|humidity|               icon|ozone|precipIntensity|precipProbability|precipType|pressure|            summary|temperature|      time|uvIndex|visibility|windBearing|windGust|windSpeed|             f_time|    w_date|w_hour|
+-------------------+----------+--------+--------+-------------------+-----+---------------+-----------------+----------+--------+-------------------+-----------+----------+-------+----------+-----------+--------+---------+-------------------+----------+------+
|              53.33|      0.93|   48.75|    0.84|               rain|306.5|         0.0474|             0.87|      rain|  1020.0|         Light Rain|      53.33|1579248000|      0|      4.94|        118|    9.85| 

Now all that's left to do is inner join the weather data on both the date and hour columns.

In [49]:
df_arr_wthr = df_arr_fmtd.join(df_weath, (col('date')==col('w_date'))&(col('hour')==col('w_hour')), how='inner')
df_arr_wthr = df_arr_wthr.drop('time', 'w_date', 'w_hour')
df_arr_wthr.show(5)

+--------------------+--------------------+-------------+--------------------+----------------------+--------------------+------------------+---+--------------------+-------------------+----------+----+------+----------+---------+-----------------------+-------------------------+-------------------------+---------------------------+----------+-----------------------+--------------------------+---------+------------+-------------------+---------------------+----------------------+-----------------------+--------------+-------------+-------------------+----------+--------+--------+-------------------+-----+---------------+-----------------+----------+--------+-------------+-----------+-------+----------+-----------+--------+---------+-------------------+
|arrivalAirportFsCode|         arrivalDate|carrierFsCode|              delays|departureAirportFsCode|     flightDurations|   flightEquipment| id|           localDate|     localTimeStamp|      date|hour|minute|weekOfYear|dayOfWeek|arrival

In [50]:
df_dep_wthr = df_dep_fmtd.join(df_weath, (col('date')==col('w_date')) & (col('hour')==col('w_hour')), how='inner')
df_dep_wthr = df_dep_wthr.drop('time', 'w_date', 'w_hour')
df_dep_wthr.show(5)

+--------------------+--------------------+-------------+----------------+----------------------+-----------------+------------------+---+--------------------+-------------------+----------+----+------+----------+---------+-----------------------+-------------------------+-------------------------+---------------------------+----------+-----------------------+--------------------------+---------+-------------------+---------------------+----------------------+-----------------------+-------------+-------------------+----------+--------+--------+-------------------+-----+---------------+-----------------+----------+--------+-------------+-----------+-------+----------+-----------+--------+---------+-------------------+
|arrivalAirportFsCode|         arrivalDate|carrierFsCode|          delays|departureAirportFsCode|  flightDurations|   flightEquipment| id|           localDate|     localTimeStamp|      date|hour|minute|weekOfYear|dayOfWeek|arrivalGateDelayMinutes|arrivalRunwayDelayMinutes

Next, I think I will select only the feature and target columns.

In [51]:
df_arr_final = df_arr_wthr.drop('arrivalDate', 'delays', 'flightDurations', 'flightEquipment', 
                               'localDate', 'localTimeStamp', 'icon', 'f_time')
df_arr_final.show(5)

+--------------------+-------------+----------------------+---+----------+----+------+----------+---------+-----------------------+-------------------------+-------------------------+---------------------------+----------+-----------------------+--------------------------+---------+------------+-------------------+---------------------+----------------------+-----------------------+--------------+-------------+-------------------+----------+--------+--------+-----+---------------+-----------------+----------+--------+-------------+-----------+-------+----------+-----------+--------+---------+
|arrivalAirportFsCode|carrierFsCode|departureAirportFsCode| id|      date|hour|minute|weekOfYear|dayOfWeek|arrivalGateDelayMinutes|arrivalRunwayDelayMinutes|departureGateDelayMinutes|departureRunwayDelayMinutes|totalDelay|actualEquipmentIataCode|scheduledEquipmentIataCode|eqChanged|blockMinutes|scheduledAirMinutes|scheduledBlockMinutes|scheduledTaxiInMinutes|scheduledTaxiOutMinutes|taxiOutMinutes|

In [52]:
# rinse and repeat for the departing flights
df_dep_final = df_dep_wthr.drop('arrivalDate', 'delays', 'flightDurations', 'flightEquipment',
                                'localDate', 'localTimeStamp', 'icon', 'f_time')
df_dep_final.show(5)

+--------------------+-------------+----------------------+---+----------+----+------+----------+---------+-----------------------+-------------------------+-------------------------+---------------------------+----------+-----------------------+--------------------------+---------+-------------------+---------------------+----------------------+-----------------------+-------------+-------------------+----------+--------+--------+-----+---------------+-----------------+----------+--------+-------------+-----------+-------+----------+-----------+--------+---------+
|arrivalAirportFsCode|carrierFsCode|departureAirportFsCode| id|      date|hour|minute|weekOfYear|dayOfWeek|arrivalGateDelayMinutes|arrivalRunwayDelayMinutes|departureGateDelayMinutes|departureRunwayDelayMinutes|totalDelay|actualEquipmentIataCode|scheduledEquipmentIataCode|eqChanged|scheduledAirMinutes|scheduledBlockMinutes|scheduledTaxiInMinutes|scheduledTaxiOutMinutes|totalDuration|apparentTemperature|cloudCover|dewPoint|hu

While we have the columns narrowed down, we still have a long way to go to be ready to build a model. Since we will be using linear regression as one of the methods, we have to encode all the categorical variables.

In [70]:
df_arr_final.printSchema()

root
 |-- arrivalAirportFsCode: string (nullable = true)
 |-- carrierFsCode: string (nullable = true)
 |-- departureAirportFsCode: string (nullable = true)
 |-- id: long (nullable = false)
 |-- date: date (nullable = true)
 |-- hour: integer (nullable = true)
 |-- minute: integer (nullable = true)
 |-- weekOfYear: integer (nullable = true)
 |-- dayOfWeek: integer (nullable = true)
 |-- arrivalGateDelayMinutes: long (nullable = true)
 |-- arrivalRunwayDelayMinutes: long (nullable = true)
 |-- departureGateDelayMinutes: long (nullable = true)
 |-- departureRunwayDelayMinutes: long (nullable = true)
 |-- totalDelay: long (nullable = true)
 |-- actualEquipmentIataCode: string (nullable = true)
 |-- scheduledEquipmentIataCode: string (nullable = true)
 |-- eqChanged: integer (nullable = true)
 |-- blockMinutes: long (nullable = true)
 |-- scheduledAirMinutes: long (nullable = true)
 |-- scheduledBlockMinutes: long (nullable = true)
 |-- scheduledTaxiInMinutes: long (nullable = true)
 |-- sc

In [53]:
# let's first create a dictionary of all the carriers present in the dataframe
# get unique values from column
unique_airlines = df_arr_final.select('carrierFsCode').distinct()

# make a list of all airlines
airline_list = [row.carrierFsCode for row in unique_airlines.collect()]

# create a random sampling of integer ids
random_codes = random.sample(range(1000), len(airline_list))

# assign each random int id to an airline
airline_dict = dict(zip(airline_list, random_codes))
print(airline_dict)


{'XOJ': 59, 'CI': 62, 'BAI': 613, 'UA': 279, 'NK': 779, 'AA': 320, 'AV': 641, 'QF': 493, 'CZ': 248, 'B6': 163, 'CX': 348, 'NH': 340, 'CV': 434, 'SU': 984, 'LX': 407, 'DL': 572, 'CA': 638, 'MW': 176, 'BR': 759, 'OO': 100, 'F9': 145, 'VB': 450, 'LH': 752, 'CM': 885, 'Y4': 141, 'VS': 570, 'HA': 291, 'BA': 963, 'LP': 975, 'EI': 874, 'WS': 386, 'DI*': 795, 'LO': 484, 'MU': 574, '5X': 762, 'G4': 503, 'MF': 490, 'NZ': 835, 'CP': 270, 'QR': 550, 'OZ': 3, 'KE': 495, '1I': 398, 'AS': 755, 'AC': 655, 'QX': 251, 'K4': 369, 'TK': 338, 'PR': 704, 'TA': 966, 'SQ': 962, 'AF': 524, 'WN': 672, 'L1E': 464}


In [54]:
total_flts = df_arr_final.count()
unique_from_stn = df_arr_final.select('departureAirportFsCode').distinct()
from_stn_total = unique_from_stn.count()
total_alines = len(airline_list)
arr_airpt = df_arr_final.first().arrivalAirportFsCode

print(f'There were {total_flts} flights from {from_stn_total} different stations operated by {total_alines} different airlines arriving at {arr_airpt} in the past 24 hours.')

There were 337 flights from 123 different stations operated by 54 different airlines arriving at LAX in the past 24 hours.


Now that we have a dictionary, let's apply it to our dataframe.

In [55]:
map_exp = create_map([lit(x) for x in chain(*airline_dict.items())])

df_arr_final = df_arr_final.withColumn('a_code', map_exp.getItem(col('carrierFsCode')))
df_arr_final.show(5)

+--------------------+-------------+----------------------+---+----------+----+------+----------+---------+-----------------------+-------------------------+-------------------------+---------------------------+----------+-----------------------+--------------------------+---------+------------+-------------------+---------------------+----------------------+-----------------------+--------------+-------------+-------------------+----------+--------+--------+-----+---------------+-----------------+----------+--------+-------------+-----------+-------+----------+-----------+--------+---------+------+
|arrivalAirportFsCode|carrierFsCode|departureAirportFsCode| id|      date|hour|minute|weekOfYear|dayOfWeek|arrivalGateDelayMinutes|arrivalRunwayDelayMinutes|departureGateDelayMinutes|departureRunwayDelayMinutes|totalDelay|actualEquipmentIataCode|scheduledEquipmentIataCode|eqChanged|blockMinutes|scheduledAirMinutes|scheduledBlockMinutes|scheduledTaxiInMinutes|scheduledTaxiOutMinutes|taxiOutM

Looking through the dataframe, we have a couple more columns that needs to go under the same process to encode.

In [56]:
# originating stations
from_stn_list = [row.departureAirportFsCode for row in unique_from_stn.collect()]
random_codes_2 = random.sample(range(1001, 2000), len(from_stn_list))
from_stn_dict = dict(zip(from_stn_list, random_codes_2))
print(from_stn_dict)

{'MSY': 1360, 'GEG': 1900, 'EUG': 1261, 'OAK': 1733, 'BNE': 1189, 'MSN': 1025, 'DCA': 1702, 'QRO': 1148, 'RDM': 1764, 'PEK': 1999, 'CMH': 1752, 'CDG': 1393, 'SIN': 1030, 'IAH': 1664, 'MZT': 1457, 'SYD': 1803, 'HNL': 1985, 'SJC': 1373, 'BUF': 1800, 'PVG': 1558, 'AUS': 1273, 'MUC': 1160, 'MEX': 1877, 'ZIH': 1904, 'RNO': 1974, 'JAC': 1748, 'IPL': 1835, 'TUL': 1591, 'BOS': 1285, 'EWR': 1570, 'DOH': 1780, 'LAS': 1435, 'YEG': 1578, 'LIR': 1336, 'BCN': 1789, 'XNA': 1476, 'YYZ': 1350, 'DEN': 1287, 'BJX': 1428, 'IAD': 1013, 'BOI': 1750, 'SBA': 1026, 'SJD': 1871, 'SEA': 1051, 'MNL': 1048, 'ICN': 1559, 'MCI': 1868, 'WAW': 1532, 'CLT': 1499, 'BNA': 1649, 'PAE': 1825, 'YVR': 1501, 'CUN': 1937, 'ABQ': 1522, 'SVO': 1924, 'SDF': 1300, 'MEL': 1569, 'SCK': 1330, 'DAL': 1907, 'MRY': 1221, 'PDX': 1574, 'PVR': 1345, 'LGW': 1524, 'MIA': 1634, 'MFR': 1633, 'LHR': 1998, 'DGO': 1530, 'PRC': 1706, 'SAL': 1553, 'OKC': 1641, 'YYC': 1564, 'SMF': 1266, 'PTY': 1403, 'PHX': 1969, 'STL': 1708, 'DFW': 1052, 'MMH': 1438

In [0]:
#df_arr_final.columns

In [58]:
# aircrafts(ac)
ac_actual = df_arr_final.select('actualEquipmentIataCode').na.fill('None').distinct()
ac_sched = df_arr_final.select('scheduledEquipmentIataCode').na.fill('None').distinct()

ac_actual_list = [row.actualEquipmentIataCode for row in ac_actual.collect()]
ac_sched_list = [row.scheduledEquipmentIataCode for row in ac_sched.collect()]

ac_combo_list = list(set(ac_actual_list + ac_sched_list))

random_codes_3 = random.sample(range(2001, 3000), len(ac_combo_list))

ac_dict = dict(zip(ac_combo_list, random_codes_3))
print(ac_dict)

print(f'A total of {len(ac_combo_list)} different types of aircraft operating today.')

{'788': 2851, 'E75': 2465, '330': 2734, '319': 2999, '74Y': 2711, '73R': 2553, '75W': 2766, '346': 2563, '388': 2202, '73G': 2888, '73H': 2745, '32S': 2362, 'E7W': 2120, '789': 2997, '77L': 2640, '76W': 2102, 'None': 2539, '752': 2642, '32A': 2941, '73J': 2206, 'CRJ': 2412, '359': 2948, 'CNJ': 2400, 'CJL': 2360, '773': 2378, '320': 2882, '77X': 2772, '781': 2097, '74F': 2762, '32Q': 2077, 'CR7': 2235, '77W': 2911, '7S8': 2187, '738': 2810, 'CR2': 2319, '31B': 2032, '757': 2628, '32B': 2232, '741': 2942, '763': 2483, '73W': 2414, '739': 2315, '747': 2372, '321': 2802, '32N': 2117, '753': 2757, '77F': 2442, '76V': 2703, 'CNA': 2434, '75D': 2509, '744': 2432, 'CN2': 2953, 'PL2': 2058, '75T': 2013, '772': 2877, '74N': 2720, '332': 2569}
A total of 57 different types of aircraft operating today.


In [60]:
# precipitation
df_arr_final.select('precipType').distinct().show()


+----------+
|precipType|
+----------+
|      null|
|      rain|
+----------+



It looks like we can just drop this column altogether.

In [0]:
df_arr_final = df_arr_final.drop('precipType', 'summary')

In [62]:
df_arr_final.show(1)

+--------------------+-------------+----------------------+---+----------+----+------+----------+---------+-----------------------+-------------------------+-------------------------+---------------------------+----------+-----------------------+--------------------------+---------+------------+-------------------+---------------------+----------------------+-----------------------+--------------+-------------+-------------------+----------+--------+--------+-----+---------------+-----------------+--------+-----------+-------+----------+-----------+--------+---------+------+
|arrivalAirportFsCode|carrierFsCode|departureAirportFsCode| id|      date|hour|minute|weekOfYear|dayOfWeek|arrivalGateDelayMinutes|arrivalRunwayDelayMinutes|departureGateDelayMinutes|departureRunwayDelayMinutes|totalDelay|actualEquipmentIataCode|scheduledEquipmentIataCode|eqChanged|blockMinutes|scheduledAirMinutes|scheduledBlockMinutes|scheduledTaxiInMinutes|scheduledTaxiOutMinutes|taxiOutMinutes|totalDuration|appa

In [0]:
map_exp_2 = create_map([lit(x) for x in chain(*from_stn_dict.items())])
map_exp_3 = create_map([lit(x) for x in chain(*ac_dict.items())])

In [0]:
df_arr_final = df_arr_final.withColumn('fm_code', map_exp_2.getItem(col('departureAirportFsCode')))
df_arr_final = df_arr_final.withColumn('aac_code', map_exp_3.getItem(col('actualEquipmentIataCode')))
df_arr_final = df_arr_final.withColumn('sac_code', map_exp_3.getItem(col('scheduledEquipmentIataCode')))

In [65]:
df_arr_final.show(5)

+--------------------+-------------+----------------------+---+----------+----+------+----------+---------+-----------------------+-------------------------+-------------------------+---------------------------+----------+-----------------------+--------------------------+---------+------------+-------------------+---------------------+----------------------+-----------------------+--------------+-------------+-------------------+----------+--------+--------+-----+---------------+-----------------+--------+-----------+-------+----------+-----------+--------+---------+------+-------+--------+--------+
|arrivalAirportFsCode|carrierFsCode|departureAirportFsCode| id|      date|hour|minute|weekOfYear|dayOfWeek|arrivalGateDelayMinutes|arrivalRunwayDelayMinutes|departureGateDelayMinutes|departureRunwayDelayMinutes|totalDelay|actualEquipmentIataCode|scheduledEquipmentIataCode|eqChanged|blockMinutes|scheduledAirMinutes|scheduledBlockMinutes|scheduledTaxiInMinutes|scheduledTaxiOutMinutes|taxiOut

Once the columns containing non-numerical values are dropped, we should finally have a dataframe we can model and predict with.

In [66]:
df_arr_num = df_arr_final.drop('arrivalAirportFsCode', 'carrierFsCode', 'departureAirportFsCode', 'date', 'id',
                               'actualEquipmentIataCode', 'scheduledEquipmentIataCode')
df_arr_num.show(5)

+----+------+----------+---------+-----------------------+-------------------------+-------------------------+---------------------------+----------+---------+------------+-------------------+---------------------+----------------------+-----------------------+--------------+-------------+-------------------+----------+--------+--------+-----+---------------+-----------------+--------+-----------+-------+----------+-----------+--------+---------+------+-------+--------+--------+
|hour|minute|weekOfYear|dayOfWeek|arrivalGateDelayMinutes|arrivalRunwayDelayMinutes|departureGateDelayMinutes|departureRunwayDelayMinutes|totalDelay|eqChanged|blockMinutes|scheduledAirMinutes|scheduledBlockMinutes|scheduledTaxiInMinutes|scheduledTaxiOutMinutes|taxiOutMinutes|totalDuration|apparentTemperature|cloudCover|dewPoint|humidity|ozone|precipIntensity|precipProbability|pressure|temperature|uvIndex|visibility|windBearing|windGust|windSpeed|a_code|fm_code|aac_code|sac_code|
+----+------+----------+--------

Just like we have been doing so far, let's process the the departing flights dataframe in a summary fashion.

In [105]:
test_rng = 1000
print(random.sample(range(test_rng), 5))

[729, 294, 652, 591, 258]


In [0]:
# function to encode each categorical variable
def encode_dict(df, column, index):
    # designate ranges for each category
    rng_str, rng_end = index*1000, (index+1)*1000

    # fill null values with 'None' and get unique values
    uniq = df.select(column).na.fill('None').distinct()

    # create a list of unique values to zip with randomized codes to create dictionary
    uniq_list = [row[column] for row in uniq.collect()]
    rnd_codes = random.sample(range(rng_str, rng_end), len(uniq_list))
    col_dict = dict(zip(uniq_list, rnd_codes))

    return col_dict

In [140]:
# function to purify joined data
def clean_combined(joined_df):
    # first gather all non-numerical data
    cat_cols = [c for c, t in joined_df.dtypes if t == 'string']

    # iterate through each column and create code dictionary
    for i, column in enumerate(cat_cols):
        print(encode_dict(joined_df, cat_cols[i], i))

clean_combined(df_arr_final)


{'HNL': 180, 'LAX': 568}
{'XOJ': 1765, 'CI': 1190, 'BAI': 1495, 'UA': 1629, 'NK': 1605, 'AA': 1224, 'AV': 1663, 'QF': 1892, 'CZ': 1285, 'B6': 1881, 'CX': 1381, 'NH': 1561, 'CV': 1837, 'SU': 1945, 'LX': 1213, 'DL': 1564, 'CA': 1065, 'MW': 1591, 'BR': 1289, 'OO': 1212, 'F9': 1019, 'VB': 1114, 'LH': 1265, 'CM': 1955, 'Y4': 1988, 'VS': 1827, 'HA': 1910, 'BA': 1467, 'LP': 1473, 'EI': 1886, 'WS': 1586, 'DI*': 1957, 'LO': 1150, 'MU': 1711, '5X': 1167, 'G4': 1622, 'MF': 1813, 'NZ': 1606, 'CP': 1526, 'QR': 1717, 'OZ': 1133, 'KE': 1336, '1I': 1873, 'AS': 1620, 'AC': 1024, 'QX': 1517, 'K4': 1391, 'TK': 1170, 'PR': 1220, 'TA': 1264, 'SQ': 1448, 'AF': 1729, 'WN': 1806, 'L1E': 1900}
{'MSY': 2241, 'GEG': 2532, 'EUG': 2422, 'OAK': 2561, 'BNE': 2814, 'MSN': 2246, 'DCA': 2856, 'QRO': 2206, 'RDM': 2586, 'PEK': 2276, 'CMH': 2022, 'CDG': 2816, 'SIN': 2825, 'IAH': 2213, 'MZT': 2553, 'SYD': 2267, 'HNL': 2410, 'SJC': 2296, 'BUF': 2638, 'PVG': 2883, 'AUS': 2712, 'MUC': 2946, 'MEX': 2870, 'ZIH': 2465, 'RNO': 28